In [1]:
import pandas as pd
import numpy as np

lane_width = 3.7
ego_width = 1.905
tolerance = 1.05*(0.5*lane_width - 0.5*ego_width)
original_excel_file = 'segment-183829460855609442_430_000_450_000_with_camera_labels.xlsx'

DataFrame = pd.read_excel ('ExcelFiles/' + original_excel_file)

ind = DataFrame.loc[(DataFrame['objY'] <= tolerance) & 
                    (DataFrame['objY'] > -1*tolerance) & 
                    (DataFrame['objSpeedY'] > -0.1) &
                    (DataFrame['objSpeedY'] < 0.1) &
                    (DataFrame['objX'] < 200) &
                    (DataFrame['objX'] > -200)]

In [2]:
following_vehicle_ID = ''
for i in range(0, 199):
    # check for the first following vehicle
    try:
        frame0_IDs = ind.loc[(ind['frame no'] == i) & (DataFrame['objX'] < 0)] 
        frame0_IDs = frame0_IDs.sort_values(by=['objX'])
        tmp = frame0_IDs['tracking ID'].iloc[-1]
        following_vehicle_ID = tmp
        print(following_vehicle_ID)
        break
    except:
        print('no following vehicle')
# getting following vehicle data
following_vehicle_data = DataFrame.loc[(DataFrame['tracking ID'] == following_vehicle_ID)]
following_vehicle_data.reset_index(drop=True, inplace=True)
# print(following_vehicle_data)

no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
no following vehicle
_eEvFUnEzuqnQxAUnEGlNQ


In [3]:
leading_vehicle_ID = ''
for i in range(0, 199):
    # check for the first following vehicle
    try:
        frame0_IDs = ind.loc[(ind['frame no'] == i) & (DataFrame['objX'] > 0)] 
        frame0_IDs = frame0_IDs.sort_values(by=['objX'])
        tmp = frame0_IDs['tracking ID'].iloc[0]
        leading_vehicle_ID = tmp
        print(leading_vehicle_ID)
        break
    except:
        print('no leading vehicle')
# getting leading vehicle data
leading_vehicle_data = DataFrame.loc[(DataFrame['tracking ID'] == leading_vehicle_ID)]
leading_vehicle_data.reset_index(drop=True, inplace=True)
# print(following_vehicle_data)

p-9IinbUxyzjqSMJImwzwQ


In [4]:
# get the acceleration of ego vehicle
len_frame = max(DataFrame['frame no'].values) + 1
egoAccX = []
tmp_acc = 0
for i in range (0, len_frame):
    try:
        segment1 = DataFrame.loc[(DataFrame['frame no'] == i)]
        segment2 = DataFrame.loc[(DataFrame['frame no'] == i+1)]
        egoSpeed1 = segment1['egoSpeedX'].iloc[0]
        egoSpeed2 = segment2['egoSpeedX'].iloc[0]
        tmp_acc = (egoSpeed2-egoSpeed1)/0.1
        egoAccX.append(tmp_acc)
    except:
        egoAccX.append(tmp_acc)
        print('end of frame')
egoAccX_df = pd.DataFrame(egoAccX, columns=['egoAccX'])
# print(egoAccX_df)

end of frame


In [5]:
export_path = 'exported_files/' + original_excel_file[:-5]

In [6]:
# for the ego and leading vehicle
# speed X of ego vehicle + speed X of leading vehicle + acceleration X of ego vehicle 
# + headway (leading -> ego)
frames_ego_leading = [leading_vehicle_data['egoSpeedX'].abs(), 
                      leading_vehicle_data['objSpeedX'].abs(), 
                      egoAccX_df['egoAccX'],
                      leading_vehicle_data['objX']
                     ]
result_ego_leading = pd.concat(frames_ego_leading, axis=1)
print(type(result_ego_leading))
result_ego_leading.to_excel(export_path + '_EgoLeading.xlsx')

<class 'pandas.core.frame.DataFrame'>


In [7]:
# for the ego vehicle and following vehicle
# speed X of following vehicle + speed X of ego vehicle + acceleration X of follower vehicle 
# + headway (ego -> following)
frames_following_ego = [following_vehicle_data['objSpeedX'].abs(), 
                      following_vehicle_data['egoSpeedX'].abs(), 
                      following_vehicle_data['objAccX'],
                      following_vehicle_data['objX']
                     ]
result_following_ego = pd.concat(frames_following_ego, axis=1)
print(type(result_following_ego))
result_following_ego.to_excel(export_path + '_FollowingEgo.xlsx')

<class 'pandas.core.frame.DataFrame'>


In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.colors as colors
# import matplotlib.cm as cmx
# %matplotlib inline

# # fig = plt.figure(figsize=(20, 10), dpi=500, facecolor='w', edgecolor='k')
# # ax = fig.add_subplot(111)
# # jet = cm = plt.get_cmap('jet') 
# # cNorm  = colors.Normalize(vmin=0, vmax=2)
# # scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

# acceleration_following = following_vehicle_data['objAccX'].to_numpy()
# acceleration_following = np.delete(acceleration_following, -1)
# # acceleration_following[acceleration_following < 0] = 0
# x_values = following_vehicle_data['objX'].to_numpy()
# x_values = abs(np.delete(x_values, -1))
# # colorVal = scalarMap.to_rgba(range(2)[1])
# # ax.plot(acceleration_following, x_values, color=colorVal, label=following_vehicle_data["tracking ID"].iloc[0])
# plt.plot(acceleration_following, x_values)
# plt.xlabel('Acceleration (m/s)', fontsize=15)
# plt.ylabel('Headway (m)', fontsize=15)
# plt.show()